## FEATURE EXTRACTION

* The main concept used here is to extract features from the images using pretrained Models and train on them. 
* To increase the generalization we can extract features using many different models, concatenate them and use them together. 
* In this way, we can achieve high accuracy even without using high end GPU's
* I have written the main code in the form of functions, so that you can use them easily.

### IMPORTING LIBRARIES

In [ ]:
# for garbage collection
import gc

# for warnings
import warnings
warnings.filterwarnings("ignore")

# utility libraries
import os
import copy
import tqdm
import numpy as np 
import pandas as pd 
import cv2, random, time, shutil, csv
import tensorflow as tf
import math

# keras libraries
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import BatchNormalization, Dense, GlobalAveragePooling2D, Lambda, Dropout, InputLayer, Input
from keras.utils import to_categorical
from keras import backend as K

### READING IMAGES

* To preserve RAM, training is done on 6000 images from the dataset. We can achieve even higher accuracy using full dataset.
* The Image size is set to 331, because NASNET requires the image size to be fixed.


In [ ]:
# set image size here
img_size = 331
data_dir = '../input/dog-breed-identification'
data_df = pd.read_csv(os.path.join(data_dir, 'labels.csv'))
class_names = sorted(data_df['breed'].unique())
print(f"No. of classes read - {len(class_names)}")
time.sleep(1)

images_list = sorted(os.listdir(os.path.join(data_dir, 'train')))
X = []
Y = []
i = 0
for image in tqdm.tqdm(images_list[:6000]):
    cls_name = data_df[data_df['id'] == image[:-4]].iloc[0,1]
    cls_index = int(class_names.index(cls_name)) 

    # Reading RGB Images
    image_path = os.path.join(data_dir, 'train',image)
    orig_image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    res_image = cv2.resize(orig_image,(img_size, img_size))
    X.append(res_image)
    Y.append(cls_index)
    i+=1

Converting the lists to arrays and deleting the list for memory saving

In [ ]:
# Converting to arrays
print(len(X), len(Y))
Xarr = np.array(X)
Yarr = np.array(Y).reshape(-1,1)

del(X)
print(Xarr.shape, Yarr.shape)
gc.collect()

One Hot encoding the classes

In [ ]:
# converting labels to one hot
Yarr_hot = to_categorical(Y)
print(Xarr.shape, Yarr_hot.shape)

Here we will prepare the training feature extraction pipeline.
* I have made separate functions for feature extraction for training and validation to experiment with augmentations for training.
* For now I have included random flip and brightness augmentations, feel free to add further.

In [ ]:
# FEATURE EXTRACTION OF TRAINING ARRAYS
AUTO = tf.data.experimental.AUTOTUNE
def get_features(model_name, data_preprocessor, data):
    '''
    1- Create a feature extractor to extract features from the data.
    2- Returns the extracted features and the feature extractor.

    '''
    dataset = tf.data.Dataset.from_tensor_slices(data)


    def preprocess(x):
        x = tf.image.random_flip_left_right(x)
        x = tf.image.random_brightness(x, 0.5)
        return x

    ds = dataset.map(preprocess, num_parallel_calls=AUTO).batch(64)

    input_size = data.shape[1:]
    #Prepare pipeline.
    input_layer = Input(input_size)
    preprocessor = Lambda(data_preprocessor)(input_layer)

    base_model = model_name(weights='imagenet', include_top=False,
                                input_shape=input_size)(preprocessor)

    avg = GlobalAveragePooling2D()(base_model)
    feature_extractor = Model(inputs = input_layer, outputs = avg)


    #Extract feature.
    feature_maps = feature_extractor.predict(ds, verbose=1)
    print('Feature maps shape: ', feature_maps.shape)
    
    # deleting variables
    del(feature_extractor, base_model, preprocessor, dataset)
    gc.collect()
    return feature_maps

In [ ]:
# FEATURE EXTRACTION OF VALIDAION AND TESTING ARRAYS
def get_valfeatures(model_name, data_preprocessor, data):
    '''
    Same as above except not image augmentations applied.
    Used for feature extraction of validation and testing.
    '''

    dataset = tf.data.Dataset.from_tensor_slices(data)

    ds = dataset.batch(64)

    input_size = data.shape[1:]
    #Prepare pipeline.
    input_layer = Input(input_size)
    preprocessor = Lambda(data_preprocessor)(input_layer)

    base_model = model_name(weights='imagenet', include_top=False,
                                input_shape=input_size)(preprocessor)

    avg = GlobalAveragePooling2D()(base_model)
    feature_extractor = Model(inputs = input_layer, outputs = avg)
    #Extract feature.
    feature_maps = feature_extractor.predict(ds, verbose=1)
    print('Feature maps shape: ', feature_maps.shape)
    return feature_maps

As explained previously, we will use several Pretrained models to extract features and concatenate them to make the final feature vector. 
* This function takes the models, preprocessors and the training/validation tag and calls the above functions.
* We can even add custom features here like, mean, variance and other statistical measures.
* This helps keep the pipeline neat

In [ ]:
# RETURNING CONCATENATED FEATURES USING MODELS AND PREPROCESSORS
def get_concat_features(feat_func, models, preprocs, array):

    print(f"Beggining extraction with {feat_func.__name__}\n")
    feats_list = []

    for i in range(len(models)):
        
        print(f"\nStarting feature extraction with {models[i].__name__} using {preprocs[i].__name__}\n")
        # applying the above function and storing in list
        feats_list.append(feat_func(models[i], preprocs[i], array))

    # features concatenating
    final_feats = np.concatenate(feats_list, axis=-1)
    # memory saving
    del(feats_list, array)
    gc.collect()

    return final_feats


* We are using 4 models here, InceptionV3, Xception, NASNetLarge and InceptionResnetV2. We are importing their model architecture and their preprocessors.
* Feel free to test with other models

In [ ]:
# DEFINING models and preprocessors imports 

from keras.applications.inception_v3 import InceptionV3, preprocess_input
inception_preprocessor = preprocess_input

from keras.applications.xception import Xception, preprocess_input
xception_preprocessor = preprocess_input

from keras.applications.nasnet import NASNetLarge, preprocess_input
nasnet_preprocessor = preprocess_input

from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
inc_resnet_preprocessor = preprocess_input

models = [InceptionV3,  InceptionResNetV2, Xception, ]
preprocs = [inception_preprocessor,  inc_resnet_preprocessor, 
            xception_preprocessor, ]

Finally the feature extraction part

In [ ]:
# calculating features of the data

final_train_features = get_concat_features(get_features, models, preprocs, Xarr)

#del(x_train, )
gc.collect()
print('Final feature maps shape', final_train_features.shape)

## MODEL TRAINING
* We are using a STRATIFIED 3 FOLD Split to train a small Deep Neural network.
* The DNN consists of the Input Layer, a Dropout for regularization and the Output Dense layer.
* The models are trained for 80 epochs with Early stopping callback and adam optimizer.
* The Models are stored in a list for prediction

In [ ]:
from keras.callbacks import EarlyStopping
EarlyStop_callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True,
                                                   verbose=0)

my_callback=[EarlyStop_callback]

In [ ]:
from sklearn.model_selection import StratifiedKFold

splits = list(StratifiedKFold(n_splits=3, shuffle=True, random_state=10).split(final_train_features, Y))

trained_models = []
accuracy = []
losses = []

#Prepare And Train DNN model

for i, (train_idx, valid_idx) in enumerate(splits): 

    print(f"\nStarting fold {i+1}\n")
    x_train_fold = final_train_features[train_idx, :]
    y_train_fold = Yarr_hot[train_idx, :]
    x_val_fold = final_train_features[valid_idx]
    y_val_fold = Yarr_hot[valid_idx, :]

    dnn = keras.models.Sequential([
        InputLayer(final_train_features.shape[1:]),
        Dropout(0.7),
        Dense(120, activation='softmax')
    ])

    dnn.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

    print("Training...")
    #Train simple DNN on extracted features.
    h = dnn.fit(x_train_fold, y_train_fold,
                batch_size=128,
                epochs=80,
                verbose=0,
                validation_data = (x_val_fold, y_val_fold),
                callbacks=my_callback)  # max 95.07

    print("Evaluating model ...")
    model_res = dnn.evaluate(x_val_fold, y_val_fold)

    accuracy.append(model_res[1])
    losses.append(model_res[0])
    trained_models.append(dnn)

print('\n CV Score -')
print(f"\nAccuracy - {sum(accuracy)/len(accuracy)}")
print(f"\nLoss - {sum(losses)/len(losses)}")

In [ ]:
# SAVING RAM

del(final_train_features, Y, Yarr_hot, Xarr)
gc.collect()

## Prediction
* Uncomment the code for prediction

In [ ]:
# TEST IMAGES
#test_images_list = sorted(os.listdir(os.path.join(data_dir, 'test')))
#X = []
#i = 0
#for image in tqdm.tqdm(test_images_list):

#    image_path = os.path.join(data_dir, 'test',image)
#    orig_image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
#    res_image = cv2.resize(orig_image,(img_size, img_size))
#    X.append(res_image)
#    i+=1

#Xtesarr = np.array(X)

#del(X)
#gc.collect()

#Xtesarr.shape

In [ ]:
# FEATURE EXTRACTION OF TEST IMAGES
#test_features = get_concat_features(get_valfeatures, models, preprocs, Xtesarr)

#del(Xtesarr)
#gc.collect()
#print('Final feature maps shape', test_features.shape)

In [ ]:
#y_pred_norm = trained_models[0].predict(test_features, batch_size=128)/3
#for dnn in trained_models[1:]:
#    y_pred_norm += dnn.predict(test_features, batch_size=128)/3

#y_pred_norm.shape

#df.iloc[:, 1:] = y_pred_norm
#df.to_csv('submission.csv')

## The END

If you have any suggestions or advice for me or for the notebook, do comment. I will be glad to hear you.
